In [3]:
import pandas as pd

# 讀取 CSV 檔案
# df = pd.read_csv('./articles.csv', encoding='utf-8')
df = pd.read_csv('./articles_584.csv', encoding='ISO-8859-1')

# 只取 content 欄位
contents = df['ARTICLE_TEXT'].tolist()

# 取前 4 篇文章
articles = {f'Article_{i+1}': content for i, content in enumerate(contents[:4])}

# # 取代 prompt 內的佔位符
# prompt_with_articles = {title: prompt + content for title, content in articles.items()}

# print(len(prompt_with_articles))  # 應該會輸出 4

In [31]:
# ==========================================
# Colab 程式碼區塊 1: 初始化和步驟1
# ==========================================

import os
import pandas as pd
from openai import OpenAI
import json
from typing import Dict, List

# 設定 OpenAI API Key (請替換成你的 API Key)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def get_response(prompt: str, temperature: float = 0.3) -> str:
    """呼叫OpenAI API"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API調用錯誤: {e}")
        return ""

# 讀取文章數據
print("📖 讀取文章數據...")
df = pd.read_csv("./articles_584.csv", encoding="ISO-8859-1")
contents = df["ARTICLE_TEXT"].dropna().tolist()
articles = {f"Article_{i+1}": content for i, content in enumerate(contents[:10])}

print(f"成功讀取 {len(articles)} 篇文章")

# Step 1 的提示詞
step1_prompt = """You are a sociology professor with 30 years of experience analyzing Asian racism in the United States.
Your task is to analyze the following article by identifying named entities and classifying them into the appropriate social roles and institutional categories. Focus especially on people or groups reacting to or affected by anti-Asian incidents.

Step 1: Named Entity Recognition and Categorization

1. Identify named entities.
2. Classify them into appropriate social roles or institutional categories.
3. Determine whether each **individual** is **Asian**, **Non-Asian**, or **Cannot be inferred** based on the text.
4. For each entity, include a `"reference"` field that reflects **the exact phrase or wording** used in the article to refer to them.

Please extract all named entities from the text and categorize them according to the following schema:

---

**INDIVIDUALS** (Specific persons or actors representing individual agency)

1. politicians
   - Elected officials acting in an individual capacity.
   - Examples: senators, representatives, mayors, governors

2. professionals
   - Individuals recognized by their expertise or institutional role.
   - Examples: professors, doctors, lawyers, foundation presidents

3. celebrities
   - Public figures in entertainment or sports (e.g., actors, athletes) unless clearly acting in a professional or political role.
   - If overlapping with another role, assign to the more institutionally grounded category.

4. perpetrators
   - Individuals directly identified as committing or responsible for anti-Asian actions.
   - Do not include vague or generalized public unless clearly specified.

5. victims
   - Individuals or racial/ethnic groups explicitly targeted by anti-Asian acts.
   - Examples: “a woman attacked on the subway,” “Japanese Americans during WWII”

6. other_individuals
   - All other named or unnamed individuals who do not fall into the above categories.
   - Includes the general public, community members, business owners, or relatives (e.g., “my mom,” “a neighbor”).

---

**ORGANIZATIONS** (Named institutions or collectives)

1. law_enforcement_agencies
   - Official police or investigative institutions.
   - Examples: Chicago Police Department, FBI, local sheriff’s office

2. government_bodies
   - Government agencies, departments, or offices at any level (local/state/federal).
   - Examples: CDC, Department of Justice, City Council

3. ngo_or_advocacy_groups
   - Civil rights organizations, foundations, or advocacy nonprofits.
   - Examples: Stop AAPI Hate, Robert Wood Johnson Foundation

4. business_entities
   - Named companies, hotels, restaurants, or stores.
   - Examples: Wrap-on Tools, Edgewater Beach Hotel

5. community_groups
   - Named cultural, ethnic, or neighborhood associations.
   - Examples: Chinatown Association, Asian-American Coalition

---

**ETHNICITY INFERENCE RULES:**

- For each **individual**, determine whether they are **Asian**, **Non-Asian**, or **Cannot be inferred**.
- Use contextual clues such as ethnicity indicators, names, or explicit mentions.
- If ethnicity is ambiguous or not stated, return `"Cannot be inferred"`.

---

**ADDITIONAL INSTRUCTIONS:**

- Use `"reference"` to capture how the person/group was referred to in the original article (e.g., `"an 80-year-old woman"`, `"Lee"`, `"the attacker"`).
- Normalize all name variants to a canonical form (e.g., “Dr. Church,” “J. Church,” and “Church” → “Jacqueline Church”).
- If an individual belongs to multiple roles, assign them to the most institutionally specific one (e.g., categorize a lawyer-celebrity as a professional).
- Include only individuals explicitly involved in specific incidents under “victims” and “perpetrators.”
- Do not classify individual police officers or sheriffs as individuals—assign them under law_enforcement_agencies.
- Classify individual owners under “business_actors” and company names under “business_entities.”

---

**Output format (in JSON):**

For all individuals, return an object with "name" and "asian_status" fields.
For all organizations, return an object with "name" and "asian_status": "Not applicable".

```json
{
  "individuals": {
    "politicians": [
      {
        "name": "Tammy Duckworth",
        "reference": "Senator Tammy Duckworth",
        "asian_status": "Asian"
      },
      {
        "name": "Joe Biden",
        "reference": "President Joe Biden",
        "asian_status": "Non-Asian"
      }
    ],
    "professionals": [
      {
        "name": "Julie Morita",
        "reference": "Julie Morita",
        "asian_status": "Asian"
      }
    ],
    "celebrities": [
      {
        "name": "Awkwafina",
        "reference": "Awkwafina",
        "asian_status": "Asian"
      }
    ],
    "journalists_or_media": [
      {
        "name": "Michelle Ye Hee Lee",
        "reference": "Lee",
        "asian_status": "Asian"
      }
    ],
    "perpetrators": [
      {
        "name": "Unknown Attacker",
        "reference": "the attacker",
        "asian_status": "Non-Asian"
      }
    ],
    "victims": [
      {
        "name": "Asian Elderly Woman",
        "reference": "an 80-year-old woman",
        "asian_status": "Asian"
      }
    ],
    "general_public": [
      {
        "name": "Bystanders",
        "reference": "bystanders",
        "asian_status": "Cannot be inferred"
      }
    ],
    "business_actors": [
      {
        "name": "Store Owner",
        "reference": "the store owner",
        "asian_status": "Asian"
      }
    ]
  },
  "organizations": {
    "law_enforcement_agencies": [
      {
        "name": "Chicago Police Department",
        "reference": "Chicago Police Department",
        "asian_status": "Not applicable"
      }
    ],
    "government_bodies": [
      {
        "name": "City Council",
        "reference": "City Council",
        "asian_status": "Not applicable"
      }
    ],
    "ngo_or_advocacy_groups": [
      {
        "name": "Stop AAPI Hate",
        "reference": "Stop AAPI Hate",
        "asian_status": "Not applicable"
      }
    ],
    "business_entities": [
      {
        "name": "Edgewater Beach Hotel",
        "reference": "Edgewater Beach Hotel",
        "asian_status": "Not applicable"
      }
    ],
    "community_groups": [
      {
        "name": "Chinatown Association",
        "reference": "Chinatown Association",
        "asian_status": "Not applicable"
      }
    ]
  }
}


"""


# 執行步驟1
print("\n🔍 執行步驟1: 命名實體識別和分類")
step1_results = {}

for title, content in articles.items():
    print(f"\n處理 {title}...")
    full_prompt = step1_prompt + "\n\nArticle Text:\n" + content
    response = get_response(full_prompt)
    step1_results[title] = response
    print(f"✅ {title} 步驟1完成")

# 顯示結果
for title, result in step1_results.items():
    print(f"\n📋 {title} - 步驟1結果:")
    print("="*60)
    print(result)
    print("="*60)

# print("\n🎉 步驟1 全部完成！")

📖 讀取文章數據...
成功讀取 10 篇文章

🔍 執行步驟1: 命名實體識別和分類

處理 Article_1...
✅ Article_1 步驟1完成

處理 Article_2...
✅ Article_2 步驟1完成

處理 Article_3...
✅ Article_3 步驟1完成

處理 Article_4...
✅ Article_4 步驟1完成

處理 Article_5...
✅ Article_5 步驟1完成

處理 Article_6...
✅ Article_6 步驟1完成

處理 Article_7...
✅ Article_7 步驟1完成

處理 Article_8...
✅ Article_8 步驟1完成

處理 Article_9...
✅ Article_9 步驟1完成

處理 Article_10...
✅ Article_10 步驟1完成

📋 Article_1 - 步驟1結果:
```json
{
  "individuals": {
    "politicians": [],
    "professionals": [
      {
        "name": "Julie Morita",
        "reference": "Julie Morita, M.D.",
        "asian_status": "Asian"
      }
    ],
    "celebrities": [],
    "perpetrators": [],
    "victims": [
      {
        "name": "Japanese Americans",
        "reference": "Japanese Americans",
        "asian_status": "Asian"
      },
      {
        "name": "Asian Americans",
        "reference": "Asian Americans",
        "asian_status": "Asian"
      }
    ],
    "other_individuals": [
      {
        "name": "P

In [33]:
# ==========================================
# Colab 程式碼區塊 2: 步驟2 - 提取相關句子
# ==========================================

# Step 2 的提示詞
step2_prompt = """You are a sociology professor with 30 years of experience analyzing Asian racism in the United States.

**Step 2: Extract Related Sentences**

Based on the named entities identified in **Step 1**, extract **all relevant complete sentences** from the text for each entity.

**For each named entity:**
1. Use the `"reference"` field (the phrase as it appears in the article) to identify relevant sentences.
2. Identify **all complete, verbatim sentences** in the text that mention or describe the entity’s **involvement, action, reaction, statement, or experience** related to anti-Asian hate (directly or indirectly).
3. Do **not paraphrase** or summarize. Use the **exact wording** from the text.
4. If no relevant sentence is found, set `"relevant_sentences": []`.
5. Return results **grouped by entity name**, exactly matching the names used in Step 1.
6. Include the following structured metadata for each entity:
   - `"entity_type"`:  
     - For individuals: the **social role** (e.g., "politician", "celebrity", "victim")  
     - For organizations: the **institutional category** (e.g., "law_enforcement_agency", "ngo_or_advocacy_group")
   - `"asian_status"`:  
     - For individuals: **"Asian"**, **"Non-Asian"**, or **"Cannot be inferred"**  
     - For organizations: Always **"Not applicable"**

**Note:** These sentences will later be used to infer **behavioral reactions** and **emotional responses**, so include **any sentence** that provides context about what the entity did, said, or experienced.

### Output format (JSON):

```json
{
  "Entity Name": {
    "entity_type": "e.g., politician, law_enforcement_agency",
    "asian_status": "Asian / Non-Asian / Cannot be inferred / Not applicable",
    "relevant_sentences": [
      "Sentence 1 from the article.",
      "Sentence 2 from the article."
    ]
  }
}

"""

# 執行步驟2
print("\n🔍 執行步驟2: 提取相關句子")
step2_results = {}

for title, content in articles.items():
    print(f"\n處理 {title}...")

    # 獲取步驟1的結果
    step1_result = step1_results[title]

    # 構建完整提示詞
    full_prompt = (step2_prompt +
                  f"\n\nStep 1 Results:\n{step1_result}" +
                  f"\n\nOriginal Article Text:\n{content}")

    response = get_response(full_prompt)
    step2_results[title] = response
    print(f"✅ {title} 步驟2完成")

# 顯示結果
for title, result in step2_results.items():
    print(f"\n📋 {title} - 步驟2結果:")
    print("="*60)
    print(result)
    print("="*60)

print("\n🎉 步驟2 全部完成！")


🔍 執行步驟2: 提取相關句子

處理 Article_1...
✅ Article_1 步驟2完成

處理 Article_2...
✅ Article_2 步驟2完成

處理 Article_3...
✅ Article_3 步驟2完成

處理 Article_4...
✅ Article_4 步驟2完成

處理 Article_5...
✅ Article_5 步驟2完成

處理 Article_6...
✅ Article_6 步驟2完成

處理 Article_7...
✅ Article_7 步驟2完成

處理 Article_8...
✅ Article_8 步驟2完成

處理 Article_9...
✅ Article_9 步驟2完成

處理 Article_10...
✅ Article_10 步驟2完成

📋 Article_1 - 步驟2結果:
```json
{
  "Julie Morita, M.D.": {
    "entity_type": "professional",
    "asian_status": "Asian",
    "relevant_sentences": [
      "Julie Morita, M.D., is executive vice president of the Robert Wood Johnson Foundation in Princeton, New Jersey.",
      "She was formerly a medical director, the chief medical officer and then commissioner for the Chicago Department of Public Health for nearly two decades."
    ]
  },
  "Japanese Americans": {
    "entity_type": "victim",
    "asian_status": "Asian",
    "relevant_sentences": [
      "As Japanese Americans, they experienced as children the pain of inter

In [34]:
# 讀取 concept tree
with open("./individual.txt", "r", encoding="utf-8") as f:
    individual_tree = f.read()
with open("./organization.txt", "r", encoding="utf-8") as f:
    organization_tree = f.read()

# 合併進 prompt
concept_trees = individual_tree + "\n\n" + organization_tree


In [35]:
# ==========================================
# Colab 程式碼區塊 3: 步驟3 - 推斷反應
# ==========================================

# Step 3 的提示詞（包含完整的概念樹）
step3_prompt = """You are a sociology professor with 30 years of experience analyzing racial dynamics and anti-Asian racism in the United States.

**Step 3: Infer Reactions Only (Do NOT infer emotions)**

Your task is to analyze the **extracted sentences** from **Step 2** and infer the **reaction** of each named entity toward anti-Asian hate, based strictly on the provided **Reaction Concept Tree**.

You will also receive metadata from Step 1, including:
- `entity_type`: the entity’s **social role or organizational category** (e.g., "politician", "law_enforcement_agency", "ngo_or_advocacy_group")
- `asian_status`: 
  - For individuals: "Asian", "Non-Asian", or "Cannot be inferred"
  - For organizations: Always "Not applicable"
  
You must use only the sentences from `relevant_sentences` in Step 2 as your source of evidence — do **not** add paraphrasing or your own interpretation.

## **Reaction Concept Tree**
{concept_trees}

---

## **Instructions**
1. Focus only on **observable reactions**: concrete actions or inactions in response to anti-Asian racism (e.g., speaking up publicly, passing legislation, ignoring complaints).
2. **Do NOT infer or mention emotions** in this step. Do not include any `emotion` field.
3. Use **only the exact extracted sentence(s)** from Step 2 as your evidence — do not paraphrase or invent explanations.
4. If the extracted sentence(s) do not describe any clear reaction, return: `"reaction": "Cannot be inferred"`.
5. The `"reaction_reason"` must always be the **exact extracted sentence(s)**.

---

## **Output format (JSON)**

```json
{
  "Entity Name": {
    "entity_type": "e.g., politician, law_enforcement_agency",
    "asian_status": "Asian / Non-Asian / Cannot be inferred / Not applicable",
    "reaction": "Reaction from the concept tree, or 'Cannot be inferred'",
    "reaction_reason": "Exact sentence(s) from Step 2"
  }
}


EXAMPLES (with their extracted sentences):

Extracted Sentence:
"Actor Daniel Dae Kim took to Instagram to condemn the recent surge in violence against Asian Americans and urged his followers to stay united."

{{
  "Daniel Dae Kim": {{
    "entity_type": "celebrity",
    "asian_status": "Asian",
    "reaction": "Speaking up on social media",
    "reaction_reason": "Actor Daniel Dae Kim took to Instagram to condemn the recent surge in violence against Asian Americans and urged his followers to stay united."
  }}
}}

Extracted Sentence:
"The victim reported the attack to local police, but no report was filed and the officers told her it wasn't a hate crime."

{{
  "local police": {{
    "entity_type": "law_enforcement_agency",
    "asian_status": "Not applicable",
    "reaction": "Did not take reports seriously",
    "reaction_reason": "The victim reported the attack to local police, but no report was filed and the officers told her it wasn't a hate crime."
  }}
}}"""

# 執行步驟3
print("\n🔍 執行步驟3: 推斷反應")
step3_results = {}

for title, content in articles.items():
    print(f"\n處理 {title}...")

    # 獲取步驟2的結果
    step2_result = step2_results[title]

    # 構建完整提示詞
    full_prompt = (step3_prompt +
                  f"\n\nStep 2 Results (Extracted Sentences):\n{step2_result}" +
                  f"\n\nOriginal Article Text (for reference):\n{content}")

    response = get_response(full_prompt)
    step3_results[title] = response
    print(f"✅ {title} 步驟3完成")

# 顯示結果
for title, result in step3_results.items():
    print(f"\n📋 {title} - 步驟3結果:")
    print("="*60)
    print(result)
    print("="*60)

print("\n🎉 步驟3 全部完成！")


🔍 執行步驟3: 推斷反應

處理 Article_1...
✅ Article_1 步驟3完成

處理 Article_2...
✅ Article_2 步驟3完成

處理 Article_3...
✅ Article_3 步驟3完成

處理 Article_4...
✅ Article_4 步驟3完成

處理 Article_5...
✅ Article_5 步驟3完成

處理 Article_6...
✅ Article_6 步驟3完成

處理 Article_7...
✅ Article_7 步驟3完成

處理 Article_8...
✅ Article_8 步驟3完成

處理 Article_9...
✅ Article_9 步驟3完成

處理 Article_10...
✅ Article_10 步驟3完成

📋 Article_1 - 步驟3結果:
```json
{
  "Julie Morita, M.D.": {
    "entity_type": "professional",
    "asian_status": "Asian",
    "reaction": "Cannot be inferred",
    "reaction_reason": "Julie Morita, M.D., is executive vice president of the Robert Wood Johnson Foundation in Princeton, New Jersey. She was formerly a medical director, the chief medical officer and then commissioner for the Chicago Department of Public Health for nearly two decades."
  },
  "Japanese Americans": {
    "entity_type": "victim",
    "asian_status": "Asian",
    "reaction": "Cannot be inferred",
    "reaction_reason": "As Japanese Americans, they expe

In [36]:
# ==========================================
# Colab 程式碼區塊 4: 步驟4 - 推斷情緒
# ==========================================

# Step 4 的提示詞（包含完整的概念樹）
step4_prompt = """You are a sociology professor with 30 years of experience analyzing racial dynamics and anti-Asian racism in the United States.

### Step 4: Infer **Emotions and Their Intensity**

Your task is to analyze the **extracted sentences** from **Step 2** and infer each entity's **emotional stance** toward anti-Asian hate.

You will also receive metadata from Step 2, including:
- `entity_type`:
   - For individuals: combine their `social_role` and `asian_status` → e.g., `"politician"`
   - For organizations: use `institutional_category` directly → e.g., `"law_enforcement_agency"`
- `asian_status`: `"Asian"`, `"Non-Asian"`, `"Cannot be inferred"`, or `"Not applicable"`

Use **only the exact `relevant_sentences`** from Step 2 as your source — **do NOT paraphrase or add your own wording**.

---

## **Emotion Concept Tree (Shaver et al., 1987)**  
Below is the full emotion tree:

- **Love**
  - Affection
    - liking
    - caring
    - compassion
    - tenderness
    - warmth
    - fondness
  - Lust
    - desire
    - passion
    - infatuation
  - Longing
    - yearning
    - nostalgia
    - homesickness

- **Joy**
  - Cheerfulness
    - amusement
    - delight
    - elation
    - excitement
    - exhilaration
    - glee
    - happiness
    - joy
    - jubilation
    - satisfaction
    - thrill
    - zest
  - Zest
    - enthusiasm
    - eagerness
    - excitement
  - Contentment
    - pleasure
    - relaxation
  - Pride
    - triumph
    - success
    - self-confidence
  - Relief
    - calm
    - eased tension

- **Anger**
  - Irritation
    - agitation
    - aggravation
    - annoyance
    - grouchiness
    - grumpiness
  - Exasperation
    - frustration
  - Rage
    - fury
    - hostility
    - violence
  - Disgust
    - revulsion
    - contempt
    - loathing
  - Envy
    - jealousy

- **Sadness**
  - Suffering
    - agony
    - hurt
  - Disappointment
    - dismay
    - letdown
  - Shame
    - guilt
    - embarrassment
    - humiliation
    - remorse
  - Neglect
    - isolation
    - rejection
  - Sympathy
    - pity
    - compassion
    - soft-heartedness

- **Fear**
  - Alarm
    - shock
    - fright
    - horror
    - terror
  - Anxiety
    - nervousness
    - worry
    - uneasiness
    - apprehension
  - Insecurity
    - embarrassment
    - shyness
    - helplessness
  - Panic
    - dread

- **Surprise**
  - Amazement
    - astonishment
  - Wonder
    - awe



---

## **Instructions**

## Instructions

1. Focus only on emotions — do NOT infer actions or behaviors.
2. Always match each sentence’s emotional meaning with the deepest word in the tree first.
   - You must select the **deepest matching term** in the concept tree — e.g., use `"rage"`, not just `"Anger"`.
   - If no match exists at the deepest level, move up one level **only if truly no leaf-level word fits**.
   - Do not invent new emotion terms.
   - If no emotion is expressed at all, label `"emotion"` as `"Cannot be inferred"` and `"emotion_path"` as `null`.
   - NEVER describe levels as "primary", "secondary", or "tertiary".
3. Always output the `emotion_path` field:
   - For tree matches: show the full path like `"Anger → Rage → fury"`.
   - Incorrect example: `"emotion": "disgust", "emotion_path": "Anger → Disgust → contempt"`
   - For `"Cannot be inferred"`: output `Cannot be inferred`.
4. If multiple distinct emotions appear in the same sentence, list each as a separate object.

5. For each emotion, label its intensity as one of the following:
    - **Neutral**: No clear emotion is expressed. The sentence is objective or emotionally flat.  
      - _Example_: “The event happened on March 16.”  
      - _Example_: “She gave a speech at the rally.”
      - _Example_: “This situation needs attention.”
    
    - **Low**: Emotion is subtle, mild, or implied with a soft tone.  
      - _Example_: “I feel a bit uneasy about this.”  
      - _Example_: “It bothers me slightly, but I’m not surprised.”
      - _Example_: “The situation is concerning.”
    
    - **Moderate**: Emotion is clearly present, but not intense or overwhelming.  
      - _Example_: “I’m disappointed by how they handled the situation.”  
      - _Example_: “This is frustrating and shouldn’t have happened.”
    
    - **High**: Emotion is strong, intense, or expressed with urgency or emphasis.  
      - _Example_: “I’m outraged! This is unacceptable!”  
      - _Example_: “I cried when I saw what happened. I’m heartbroken.”

    - Use the examples above to guide your decisions. If no emotion is present, label it as **Neutral**.

6. Use the exact sentence(s) from Step 2 as your `"emotion_reason"` — do NOT paraphrase.
7. If no emotion is expressed at all, return:
   ```json
   "emotions": [
     {
       "emotion": "Cannot be inferred",
       "emotion_path": null,
       "emotion_intensity": null,
       "emotion_reason": "Exact sentence(s) from Step 2"
     }
   ]```
7. Think step by step:
   - Analyze the meaning.
   - Find the best subtree.
   - Select the deepest applicable term.
---

## **Output format (JSON)**

```json
{{
  "Entity Name": {{
    "entity_type": "e.g., politician, law_enforcement_agency",
    "asian_status": "Asian / Non-Asian / Cannot be inferred / Not applicable",
    "emotions": [
      {{
        "emotion": "the deepest matched motion term from concept tree",
        "emotion_path": "Path in tree or null",
        "emotion_intensity": "low / moderate / high",
        "emotion_reason": "Exact sentence(s) from Step 2"
      }},
      ...
    ]
  }}
}}
```


EXAMPLES (with their extracted sentences):

Extracted Sentence:
"Actor Daniel Dae Kim expressed outrage and sadness over the recent surge in violence against Asian Americans."

{{
  "Daniel Dae Kim": {{
    "entity_type": "celebrity",
    "asian_status": "Asian",
    "emotions": [
      {{
        "emotion": "fury",
        "emotion_path": "Anger → Rage → fury",
        "emotion_intensity": "high",
        "emotion_reason": "Actor Daniel Dae Kim expressed outrage and sadness over the recent surge in violence against Asian Americans."
      }},
      {{
        "emotion": "sadness",
        "emotion_path": "Sadness",
        "emotion_intensity": "moderate",
        "emotion_reason": "Actor Daniel Dae Kim expressed outrage and sadness over the recent surge in violence against Asian Americans."
      }}
    ]
  }}
}}

Extracted Sentence:
"The victim reported the attack to local police, but no report was filed and the officers told her it wasn't a hate crime."

{{
  "local police": {{
    "entity_type": "law_enforcement_agency",
    "asian_status": "Not applicable",
    "emotions": [
      {{
        "emotion": "Cannot be inferred",
        "emotion_path": null,
        "emotion_intensity": null,
        "emotion_reason": "The victim reported the attack to local police, but no report was filed and the officers told her it wasn't a hate crime."
      }}
    ]
  }}
}}"""

print("\n🔍 執行步驟4: 推斷反應或情緒")
step4_results = {}

for title, content in articles.items():
    print(f"\n處理 {title}...")

    step2_result = step2_results[title]

    full_prompt = (step4_prompt +
                   f"\n\nStep 2 Results (Extracted Sentences):\n{step2_result}" +
                   f"\n\nOriginal Article Text (for reference):\n{content}")

    response = get_response(full_prompt)
    
    # ✅ 和 Step 3 一樣：不解析，直接存文字
    step4_results[title] = response
    print(f"✅ {title} 步驟4完成")


# 顯示結果
for title, result in step4_results.items():
    print(f"\n📋 {title} - 步驟4結果:")
    print("="*60)
    print(json.dumps(result, indent=2, ensure_ascii=False))  # 美化格式顯示
    print("="*60)

print("\n🎉 步驟4 全部完成！")



🔍 執行步驟4: 推斷反應或情緒

處理 Article_1...
✅ Article_1 步驟4完成

處理 Article_2...
✅ Article_2 步驟4完成

處理 Article_3...
✅ Article_3 步驟4完成

處理 Article_4...
✅ Article_4 步驟4完成

處理 Article_5...
✅ Article_5 步驟4完成

處理 Article_6...
✅ Article_6 步驟4完成

處理 Article_7...
✅ Article_7 步驟4完成

處理 Article_8...
✅ Article_8 步驟4完成

處理 Article_9...
✅ Article_9 步驟4完成

處理 Article_10...
✅ Article_10 步驟4完成

📋 Article_1 - 步驟4結果:
"```json\n{\n  \"Julie Morita, M.D.\": {\n    \"entity_type\": \"professional\",\n    \"asian_status\": \"Asian\",\n    \"emotions\": [\n      {\n        \"emotion\": \"Cannot be inferred\",\n        \"emotion_path\": null,\n        \"emotion_intensity\": null,\n        \"emotion_reason\": \"Julie Morita, M.D., is executive vice president of the Robert Wood Johnson Foundation in Princeton, New Jersey.\"\n      },\n      {\n        \"emotion\": \"Cannot be inferred\",\n        \"emotion_path\": null,\n        \"emotion_intensity\": null,\n        \"emotion_reason\": \"She was formerly a medical directo

In [41]:
import json

print("\n--- 所有文章的步驟 4 處理結果 ---")
print("=" * 60)

for title, result in step2_results.items():
    print(f"\n📋 文章 '{title}' - 步驟 4 結果:")
    print("-" * 50) # 分隔線

    if isinstance(result, dict):
        # 如果結果是字典 (已經成功解析的 JSON)
        print(json.dumps(result, indent=2, ensure_ascii=False))
    elif isinstance(result, str):
        # 如果結果是字串 (可能是未解析的原始 JSON 字串或錯誤訊息)
        print("💡 注意：此結果可能是一個未解析的 JSON 字串或原始回應。")
        print(result)
    else:
        # 其他情況
        print(f"結果類型為：{type(result)}")
        print(result)

    print("-" * 50) # 分隔線
    print("\n") # 在每個文章結果之間增加空白行，提高可讀性

print("=" * 60)
print("--- 所有結果顯示完畢 ---")


--- 所有文章的步驟 4 處理結果 ---

📋 文章 'Article_1' - 步驟 4 結果:
--------------------------------------------------
💡 注意：此結果可能是一個未解析的 JSON 字串或原始回應。
```json
{
  "Julie Morita, M.D.": {
    "entity_type": "professional",
    "asian_status": "Asian",
    "relevant_sentences": [
      "Julie Morita, M.D., is executive vice president of the Robert Wood Johnson Foundation in Princeton, New Jersey.",
      "She was formerly a medical director, the chief medical officer and then commissioner for the Chicago Department of Public Health for nearly two decades."
    ]
  },
  "Japanese Americans": {
    "entity_type": "victim",
    "asian_status": "Asian",
    "relevant_sentences": [
      "As Japanese Americans, they experienced as children the pain of internment during World War II, one of the terrible scars in our country's past that stemmed from fear, hate and racism."
    ]
  },
  "Asian Americans": {
    "entity_type": "victim",
    "asian_status": "Asian",
    "relevant_sentences": [
      "With the co

In [ ]:
## **Emotion Concept Tree (Shaver et al., 1987)**  
{{
  "love": {{
    "affection": ["liking", "caring", "compassion", "tenderness", "warmth", "fondness"],
    "lust": ["desire", "passion", "infatuation"],
    "longing": ["yearning", "nostalgia", "homesickness"]
  }},
  "joy": {{
    "cheerfulness": ["amusement", "delight", "elation", "excitement", "exhilaration", "glee", "happiness", "joy", "jubilation", "satisfaction", "thrill", "zest"],
    "zest": ["enthusiasm", "eagerness", "excitement"],
    "contentment": ["pleasure", "relaxation"],
    "pride": ["triumph", "success", "self-confidence"],
    "relief": ["calm", "eased tension"]
  }},
  "anger": {{
    "irritation": ["agitation", "aggravation", "annoyance", "grouchiness", "grumpiness"],
    "exasperation": ["frustration"],
    "rage": ["fury", "hostility", "violence"],
    "disgust": ["revulsion", "contempt", "loathing"],
    "envy": ["jealousy"]
  }},
  "sadness": {{
    "suffering": ["agony", "hurt"],
    "disappointment": ["dismay", "letdown"],
    "shame": ["guilt", "embarrassment", "humiliation", "remorse"],
    "neglect": ["isolation", "rejection"],
    "sympathy": ["pity", "compassion", "soft-heartedness"]
  }},
  "fear": {{
    "alarm": ["shock", "fright", "horror", "terror"],
    "anxiety": ["nervousness", "worry", "uneasiness", "apprehension"],
    "insecurity": ["embarrassment", "shyness", "helplessness"],
    "panic": ["dread"]
  }},
  "surprise": {{
    "amazement": ["astonishment"],
    "wonder": ["awe"]
  }}
}}

In [46]:
import json
import csv

csv_filename = "merged_output.csv"

csv_fields = [
    "id",
    "article_id",
    "entity_name",
    "entity_type",
    "asian_status",
    "reaction",
    "reaction_reason",
    "emotion",
    "emotion_path",
    "emotion_intensity",
    "emotion_reason"
]

row_id = 1

with open(csv_filename, "w", newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_fields)
    writer.writeheader()

    for article_id in step3_results.keys():
        try:
            step2 = json.loads(step2_results[article_id].strip("`json\n").strip("`"))
            step3 = json.loads(step3_results[article_id].strip("`json\n").strip("`"))
            step4 = json.loads(step4_results[article_id].strip("`json\n").strip("`"))
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON decode error for {article_id}: {e}")
            continue

        for entity_name in step2.keys():
            s2 = step2.get(entity_name, {})
            s3 = step3.get(entity_name, {})
            s4 = step4.get(entity_name, {})

            entity_type = s2.get("entity_type") or s3.get("entity_type") or s4.get("entity_type")
            asian_status = s2.get("asian_status") or s3.get("asian_status") or s4.get("asian_status")
            reaction = s3.get("reaction", "")
             # fallback for reaction_reason
            reaction_reason = s3.get("reaction_reason", "").strip()
            if not reaction_reason:
                if relevant_sentences:
                    reaction_reason = " ".join(relevant_sentences)
                else:
                    reaction_reason = f"No relevant sentence found."

            relevant_sentences = s2.get("relevant_sentences", [])

            emotions = s4.get("emotions", [])

            if not emotions:
                fallback_reason = " ".join(relevant_sentences) if relevant_sentences else f"No relevant sentence found."
                writer.writerow({
                    "id": row_id,
                    "article_id": article_id,
                    "entity_name": entity_name,
                    "entity_type": entity_type,
                    "asian_status": asian_status,
                    "reaction": reaction,
                    "reaction_reason": reaction_reason,
                    "emotion": "Cannot be inferred",
                    "emotion_path": "null",
                    "emotion_intensity": "null",
                    "emotion_reason": fallback_reason
                })
                row_id += 1

            else:
                for e in emotions:
                    # 修正 emotion 用 path 最後一節
                    emotion_value = e.get("emotion", "")
                    emotion_path = e.get("emotion_path", "")
                    if emotion_path and "→" in emotion_path:
                        last_part = emotion_path.split("→")[-1].strip()
                        if last_part:
                            emotion_value = last_part

                    # 修正 emotion_reason fallback
                    emotion_reason = e.get("emotion_reason", "").strip()
                    if not emotion_reason:
                        if relevant_sentences:
                            emotion_reason = " ".join(relevant_sentences)
                        else:
                            emotion_reason = f"No relevant sentence found."

                    writer.writerow({
                        "id": row_id,
                        "article_id": article_id,
                        "entity_name": entity_name,
                        "entity_type": entity_type,
                        "asian_status": asian_status,
                        "reaction": reaction,
                        "reaction_reason": reaction_reason,
                        "emotion": emotion_value,
                        "emotion_path": emotion_path if emotion_path is not None else "null",
                        "emotion_intensity": e.get("emotion_intensity", "null") if e.get("emotion_intensity") is not None else "null",
                        "emotion_reason": emotion_reason
                    })
                    row_id += 1


⚠️ JSON decode error for Article_3: Extra data: line 10 column 1 (char 284)


In [50]:
print("\n=== RAW Step 2 ===\n", step3_results[article_id])


=== RAW Step 2 ===
 ```json
{
  "Andrew Yang": {
    "entity_type": "politician",
    "asian_status": "Asian",
    "reaction": "Blaming the victim",
    "reaction_reason": "Mr. Yang was blaming the victim at a time when Asian Americans are at their most vulnerable position in decades."
  },
  "David Inoue": {
    "entity_type": "professional",
    "asian_status": "Asian",
    "reaction": "Cannot be inferred",
    "reaction_reason": "The writer is executive director for the Japanese American Citizens League."
  },
  "April Lee": {
    "entity_type": "professional",
    "asian_status": "Cannot be inferred",
    "reaction": "Speaking out against racism",
    "reaction_reason": "As a first-generation Chinese American, I am appalled and outraged by Andrew Yang's comments on recent racist attacks against Asian Americans."
  },
  "Alexander Vindman": {
    "entity_type": "politician",
    "asian_status": "Non-Asian",
    "reaction": "Cannot be inferred",
    "reaction_reason": "Call the roll

In [1]:
import pandas as pd

# 讀取 CSV 檔案
csv_file = 'merged_output.csv'  # 請替換成你的檔案名稱
df = pd.read_csv(csv_file)

# 將 DataFrame 寫入 Excel 檔案
excel_file = 'RQ1-2_test_result.xlsx'  # 輸出的檔名
df.to_excel(excel_file, index=False)  # index=False 表示不要把索引寫進 Excel

print(f"已將 {csv_file} 轉換為 {excel_file}")


已將 merged_output.csv 轉換為 RQ1-2_test_result.xlsx
